# Data Exploration

In [ ]:
import os
import xml.etree.ElementTree as et
import json
import ijson

In [ ]:
#posts_path = os.path.join("E:", "Uni", "DataViz", "DataSet", "Posts.xml")
#questions_path = os.path.join("E:", "Uni", "DataViz", "DataSet", "Questions.json")
#answers_path = os.path.join("E:", "Uni", "DataViz", "DataSet", "Answers.json")
#edge_list_path = os.path.join("E:", "Uni", "DataViz", "DataSet", "Edges.json")
#edge_list_tag_path = os.path.join("E:", "Uni", "DataViz", "DataSet", "Tags")

posts_path = os.path.join("Posts.xml")
questions_path = os.path.join("Questions.json")
answers_path = os.path.join("Answers.json")
edge_list_path = os.path.join("Edges.json")
edge_list_tag_path = os.path.join("Tags")

In [ ]:
posts_path

In [ ]:
with open(questions_path, 'w') as q, open(answers_path, 'w') as a:
    q.write("{ \"data\": [\n")
    a.write("{ \"data\": [\n")
    firstQ = True
    firstA = True
    for event, elem in et.iterparse(posts_path):
        if elem.tag == "row":
            try:
                parsed = {}
                parsed["id"] = elem.attrib["Id"]
                parsed["owner_id"] = elem.attrib["OwnerUserId"]
                parsed["time"] = elem.attrib["CreationDate"]
                try:                    
                    parsed["tags"] = elem.attrib["Tags"]
                except KeyError:
                    parsed["tags"] = ""
                if elem.attrib["PostTypeId"] == "1":
                    parsed["ref"] = elem.attrib["AcceptedAnswerId"]
                    if not firstQ:
                        q.write(",\n")
                    else:
                        firstQ = False
                    q.write(json.dumps(parsed))
                else :                    
                    parsed["ref"] = elem.attrib["ParentId"]
                    if not firstA:
                        a.write(",\n")
                    else:
                        firstA = False
                    a.write(json.dumps(parsed))
            except KeyError as e:
                pass
                # ignore posts without answer for now
                # ignore posts without user id
        elem.clear()
    q.write("\n]}")
    a.write("\n]}")

In [ ]:
with open(questions_path, 'r') as q, open(edge_list_path,'w') as e:
    questions = ijson.items(q, "data.item")
    first = True
    e.write("{ \"data\":[")
    for question in questions:
        with open(answers_path, 'r') as a:
            answers = ijson.items(a, "data.item")
            for answer in answers:
                if question['ref'] == answer['id']:
                    if not first:
                        e.write(",\n")
                    else:
                        first = False
                    e.write(json.dumps({'q_id':question["owner_id"], 'a_id':answer["owner_id"], 'time':answer["time"], 'tags':question["tags"]}))
                    break
    e.write("]}")

In [ ]:
for the_file in os.listdir(edge_list_tag_path):
    file_path = os.path.join(edge_list_tag_path, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)


with open(edge_list_path,'r') as e:
    edges = ijson.items(e, "data.item")
    for edge in edges:
        print (edge)
        # split tags
        tags = [s.replace("<", "").replace(">","") for s in edge["tags"].split("><")]
        print (tags)
        for tag in tags:
            with open(os.path.join(edge_list_tag_path, "{}.json".format(tag)), 'a') as e_tag:
                e_tag.write(json.dumps(edge)+"\n")
